In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv("clogit_df.csv",encoding='utf-8')
data = data.dropna()
#data = data.dropna() # 이거로 하면 cost 5자리수 나옴

y = data['record']
X = data.loc[:, ['code','lane','age','jockey_w','jockey','dandivi','yeondivi',
                 'cure_in_1m','weight_diff', 'raw_weight','weight_added','distance',
                 'prev1_velo','prev2_velo','avg_vel','adj_vel','norating', 'sex1', 'sex2']]
#prev3은 도움안됨
'''
# sex는 dummy 함수 처리
sex_mod = pd.get_dummies(X['sex'])
X = X.join(sex_mod)
X = X.drop(columns=['sex'])
#print(X.columns)
'''
X = X.as_matrix()

In [7]:

X

array([[3.4628e+04, 3.0000e+00, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.9973e+04, 1.1000e+01, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [3.9806e+04, 6.0000e+00, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [3.0853e+04, 5.0000e+00, 4.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       [2.9616e+04, 9.0000e+00, 4.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.9820e+04, 7.0000e+00, 4.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        0.0000e+00]])

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
train_size = len(X_train)
print("train_size : ",train_size)

size_x = len(X_train[0])
print("size_x : ",size_x)

# 설정하는 부분
learning_rate = 0.01
training_epochs = 500
batch_size = 128


train_size :  4804
size_x :  19


In [65]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool, name='phase')


W1 = tf.get_variable(shape=[size_x, 128], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)
layer1 = tf.contrib.layers.batch_norm(layer1, center=True, scale=True, 
                                          is_training=phase)

W2 = tf.get_variable(shape=[128, 64], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
layer2 = tf.contrib.layers.batch_norm(layer2, center=True, scale=True, 
                                          is_training=phase)

W3 = tf.get_variable(shape=[64, 32], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([32]))
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)
layer3 = tf.contrib.layers.batch_norm(layer3, center=True, scale=True, 
                                          is_training=phase)

W4 = tf.get_variable(shape=[32, 1], name='weight4', initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(layer3, W4) + b4
hypothesis = tf.contrib.layers.batch_norm(hypothesis, center=True, scale=True, 
                                          is_training=phase)


cost = tf.reduce_mean(tf.square(hypothesis - Y))
#cost = tf.losses.mean_squared_error(labels=Y, predictions=hypothesis)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        batch_xs = X_train[i*batch_size:(i+1)*batch_size]
        batch_ys = y_train[i*batch_size:(i+1)*batch_size].reshape(batch_size,1)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7,phase:True}
        c, _ = sess.run([cost, train], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    if epoch%10==0:
      print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Epoch: 0001 cost = 13837.613809122
Epoch: 0011 cost = 12972.444072002
Epoch: 0021 cost = 12171.815376900
Epoch: 0031 cost = 11385.225955448
Epoch: 0041 cost = 10643.093327703
Epoch: 0051 cost = 9937.804793074
Epoch: 0061 cost = 9257.140677787
Epoch: 0071 cost = 8611.694811022
Epoch: 0081 cost = 7990.397540118
Epoch: 0091 cost = 7397.058501372
Epoch: 0101 cost = 6834.533440667
Epoch: 0111 cost = 6290.615788640
Epoch: 0121 cost = 5781.484678526
Epoch: 0131 cost = 5290.448585304
Epoch: 0141 cost = 4823.134567673
Epoch: 0151 cost = 4379.707664696
Epoch: 0161 cost = 3961.866098765
Epoch: 0171 cost = 3567.706430796
Epoch: 0181 cost = 3208.666734850
Epoch: 0191 cost = 2859.875673036
Epoch: 0201 cost = 2533.799065667
Epoch: 0211 cost = 2231.685797614
Epoch: 0221 cost = 1951.436589421
Epoch: 0231 cost = 1693.549313107
Epoch: 0241 cost = 1456.957466744
Epoch: 0251 cost = 1241.967090504
Epoch: 0261 cost = 1046.782152331
Epoch: 0271 cost = 869.115881018
Epoch: 0281 cost = 715.078334499
Epoch: 0291

In [67]:

y_test = y_test.reshape(-1,1)
mse = tf.reduce_mean(tf.square(hypothesis - Y))
print('Accuracy:', sess.run(mse, feed_dict={
      X: X_test, Y: y_test, keep_prob:1, phase:False}))

Accuracy: 46511040000000.0
